In [3]:
import altair as alt
import pandas as pd
import numpy as np
# norms
# mean sj: (1,1,0,0,0,0,1,1)
# nice sj: (1,1,0,1,0,1,1,1)

ebnorms_b = ["(1,1,0,0,0,0,1,1)", "(1,1,0,1,0,1,1,1)", "(1,1,1,0,1,0,1,1)", "(0,0,0,0,1,1,1,1)", "(0,1,0,1,1,1,1,1)",
          "(1,1,0,0,1,1,1,1)", "(1,1,0,1,1,1,1,1)", "(0,0,0,0,0,0,1,1)", "(0,1,0,1,0,1,1,1)", "(0,0,0,1,0,1,1,1)"]

ebnorms_n = ["SJ normal", "SJ nice", "SJ Mean", "IS normal", "IS nice", 
             "SS normal", "SS nice", "SH normal", "SH normal", "IS Emp"]

norms_simple_b = ["(1,0,0,1)", "(1,0,1,1)", "(0,0,0,1)", "(0,0,1,1)"]

norms_simple_n = ["SJ", "SS", "SH", "IS"]

columns = ["EBNorm", "Norm", "z", "g", "mu", "chi", "eps", "gamma", "pdx", "acr", "B", "G", "AllD", "pDisc", "Disc", "AllC", "mean", "nice"]

def load_and_process() -> pd.DataFrame: 
    data: pd.DataFrame = pd.read_csv("outputs/results.txt", sep="\t", header=None, names=columns, index_col=False)
    data.eps = data.eps / data.z
    data.dropna()
    data.EBNorm = data.EBNorm.replace(ebnorms_b, ebnorms_n)
    data.Norm = data.Norm.replace(norms_simple_b, norms_simple_n)
    return data

def filter(frame: pd.DataFrame, feature: str, condition) -> pd.DataFrame:
    return frame[frame[feature]==condition]
    
data = load_and_process()


In [53]:
data

,EBNorm,Norm,z,g,mu,chi,eps,gamma,pdx,acr,B,G,AllD,pDisc,Disc,AllC,mean,nice
0,IS normal,IS,50.0,4000.0,0.1,0.1,0.00002,0.00,True,14.889027,1.00,0.00,1.0,0.00,0.0,0.00,0.0,1.0
1,IS normal,IS,50.0,4000.0,0.1,0.1,0.00002,0.00,True,7.046797,1.00,0.00,1.0,0.00,0.0,0.00,1.0,0.0
2,IS normal,IS,50.0,4000.0,0.1,0.1,0.00002,0.00,True,19.991373,1.00,0.00,1.0,0.00,0.0,0.00,1.0,0.0
3,IS normal,IS,50.0,4000.0,0.1,0.1,0.00002,0.00,True,14.513079,0.00,1.00,0.0,0.24,0.0,0.76,0.0,1.0
4,IS normal,IS,50.0,4000.0,0.1,0.1,0.00002,0.00,True,10.351082,1.00,0.00,1.0,0.00,0.0,0.00,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52572,IS Emp,IS,50.0,4000.0,0.1,0.1,0.20000,1.00,True,41.087530,0.08,0.92,1.0,0.00,0.0,0.00,0.0,1.0
52573,IS Emp,IS,50.0,4000.0,0.1,0.1,0.20000,1.00,True,37.628362,0.78,0.22,1.0,0.00,0.0,0.00,1.0,0.0
52574,IS Emp,IS,50.0,4000.0,0.1,0.1,0.20000,1.00,True,20.331490,0.80,0.20,1.0,0.00,0.0,0.00,1.0,0.0
52575,IS Emp,IS,50.0,4000.0,0.1,0.1,0.20000,0.95,True,23.713826,0.10,0.90,1.0,0.00,0.0,0.00,0.0,1.0


In [49]:
chosen_norm: str = "IS nice"
filtered = data[data.EBNorm == chosen_norm]
filtered = filter(filtered, "z", 50)

new_cols = ["gamma", "eps", "acr", "g_reps", "nice_eps", "AllD", "pDisc", "Disc", "AllC"]
new_df = pd.DataFrame(columns=new_cols)

gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.05)]

for gamma in gammas:
    for epsilon in sorted(filtered.eps.unique()):
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered.eps == epsilon)]
        
        mean_acr: float = super_filtered.acr.mean()
        mean_greps: float = super_filtered.G.mean()
        mean_niceeps: float = super_filtered.nice.mean()
            
        mean_AllC: float = super_filtered.AllC.mean()
        mean_Disc: float = super_filtered.Disc.mean()
        mean_AllD: float = super_filtered.AllD.mean()
        mean_pDisc: float = super_filtered.pDisc.mean()
        new_df.loc[len(new_df)] = [gamma, epsilon, mean_acr, mean_greps, mean_niceeps, 
                                   mean_AllD, mean_pDisc, mean_Disc, mean_AllC]
            

feature: str = "AllD" + ":Q"
            
heatmap = alt.Chart(new_df).mark_rect().encode(
    x='eps:O',
    y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
    #color=alt.Color(feature, scale=alt.Scale(domain=[5,30]))
    color=feature
).properties(
    height=400,
    width=400,
    title = chosen_norm
)

text = heatmap.mark_text().encode(
    text=alt.Text(feature, format=".2f"),
    color=alt.value('black')
)

#heatmap+text

In [50]:
heatmap + text



alt.LayerChart(...)

In [65]:
data = load_and_process()
filtered = filter(data, "z", 50)

new_cols = ["gamma", "eps", "acr"]
new_df = pd.DataFrame(columns=new_cols)

gammas = [round(a, 2) for a in np.arange(0, 1.01, 0.05)]

for gamma in gammas:
    for epsilon in sorted(filtered.eps.unique()):
        super_filtered = filtered[(filtered.gamma == gamma) & (filtered.eps == epsilon)]
        #print(super_filtered.shape)
        
        mean_acr_nice: float = super_filtered[super_filtered.EBNorm=="IS Emp"].acr.mean()
        mean_acr_normal: float = super_filtered[super_filtered.EBNorm=="IS normal"].acr.mean()
        new_df.loc[len(new_df)] = [gamma, epsilon, mean_acr_nice-mean_acr_normal]
        
        
        
feature: str = "acr" + ":Q"
            
heatmap = alt.Chart(new_df).mark_rect().encode(
    x='eps:O',
    y=alt.Y('gamma:O', sort=alt.EncodingSortField('gamma', order='descending')),
    color=alt.Color(feature, scale=alt.Scale(domain=[-20,20], scheme='blueorange'), sort="descending")
).properties(
    height=400,
    width=400,
    title = "ACR Difference between EB-IS and IS"
)

text = heatmap.mark_text().encode(
    text=alt.Text(feature, format=".2f"),
    color=alt.value('black')
)

heatmap + text


alt.LayerChart(...)